# 兴奋性-抑制性神经元网络（E-I网络）的模拟与分析

本教程将基于 Braincell 框架实现一个经典的兴奋性-抑制性神经元网络。通过构建由 Hodgkin-Huxley（HH）模型神经元组成的 E-I 网络，你将学习如何在 Braincell 中实现从单细胞到网络的层级建模，并分析网络的 spike 动态特性。


## 教程目标
- 掌握使用 Braincell 构建基于 HH 模型的单神经元（`SingleCompartment`）。
- 理解兴奋性-抑制性（E-I）网络的结构设计。
- 学习如何配置网络模拟参数并运行大规模网络仿真。
- 可视化网络中神经元的 spike 活动，分析 E-I 平衡下的网络动态。


## 准备工作
首先确保已安装必要的库（`braincell`、`brainstate`、`brainunit`、`matplotlib`），并导入所需模块：

```python
import brainunit as u  # 神经科学常用单位
import matplotlib.pyplot as plt  # 基础绘图库
import brainstate  # 用于网络动态管理和仿真
import braincell  # 核心库：构建神经元和离子通道
```


## 代码详解

### 参数定义：神经元与网络的物理属性
首先定义关键物理参数，这些参数决定了神经元的电生理特性和网络规模：

```python
# 神经元动作电位发放阈值
V_th = -20. * u.mV

# 神经元膜面积（转换为 cm² 便于与电导单位匹配）
area = 20000 * u.um **2  # 初始单位为平方微米
area = area.in_unit(u.cm** 2)  # 转换为平方厘米（1 cm² = 1e8 μm²）

# 膜电容（基于膜面积和比电容：1 uF/cm² 是生物膜的典型值）
Cm = (1 * u.uF * u.cm ** -2) * area  # 总电容 = 比电容 × 面积（单位：pF）
```


### 定义 HH 单神经元模型
使用 `braincell.SingleCompartment` 构建基于 HH 模型的单神经元，包含钠通道（`INa`）、钾通道（`IK`）和漏电流（`IL`），这些通道共同决定神经元的放电特性：

```python
class HH(braincell.SingleCompartment):
    def __init__(self, in_size):
        # 初始化单隔室神经元：in_size 为神经元数量（支持批量建模），C 为膜电容，solver 为积分方法
        super().__init__(in_size, C=Cm, solver='ind_exp_euler')

        # 钠离子通道（INa）：基于 Traub & Miles 1991 年的模型
        self.na = braincell.ion.SodiumFixed(in_size, E=50. * u.mV)  # 钠反转电位 50 mV
        self.na.add_elem(
            # 最大电导：100 mS/cm²（比电导）× 膜面积 → 总电导
            INa=braincell.channel.INa_TM1991(in_size, g_max=(100. * u.mS * u.cm **-2) * area, V_sh=-63. * u.mV)
        )

        # 钾离子通道（IK）：同样基于 Traub & Miles 1991 年的模型
        self.k = braincell.ion.PotassiumFixed(in_size, E=-90 * u.mV)  # 钾反转电位 -90 mV
        self.k.add_elem(
            # 最大电导：30 mS/cm² × 膜面积
            IK=braincell.channel.IK_TM1991(in_size, g_max=(30. * u.mS * u.cm** -2) * area, V_sh=-63. * u.mV)
        )

        # 漏电流（IL）：非选择性离子通道，维持静息电位
        self.IL = braincell.channel.IL(
            in_size,
            E=-60. * u.mV,  # 漏电流反转电位 -60 mV
            g_max=(5. * u.nS * u.cm **-2) * area  # 最大电导：5 nS/cm² × 膜面积
        )
```


### 定义 E-I 网络：兴奋性与抑制性神经元的连接
构建由兴奋性（E）和抑制性（I）神经元组成的网络，模拟皮层网络中常见的 E-I 平衡机制：

```python
class EINet(brainstate.nn.DynamicsGroup):
    def __init__(self):
        super().__init__()
        # 网络规模：3200 个兴奋性神经元，800 个抑制性神经元（比例 4:1，符合皮层特性）
        self.n_exc = 3200
        self.n_inh = 800
        self.num = self.n_exc + self.n_inh  # 总神经元数：4000

        # 初始化神经元群体：使用前面定义的 HH 模型，数量为总神经元数
        self.N = HH(self.num)

        # 兴奋性突触投射（E→所有神经元）
        self.E = brainstate.nn.AlignPostProj(
            # 连接规则：兴奋性神经元向所有神经元（E和I）以 2% 概率连接
            comm=brainstate.nn.EventFixedProb(
                self.n_exc, self.num, conn_num=0.02,  # conn_num 为连接概率
                conn_weight=6. * u.nS  # 突触权重（电导）：6 nS
            ),
            # 突触动力学：指数衰减突触（时间常数 5 ms，典型兴奋性突触特性）
            syn=brainstate.nn.Expon(self.num, tau=5. * u.ms),
            # 突触后效应：COBA 模型（ conductance-based 突触），反转电位 0 mV（兴奋性）
            out=brainstate.nn.COBA(E=0. * u.mV),
            post=self.N  # 投射目标为神经元群体 N
        )

        # 抑制性突触投射（I→所有神经元）
        self.I = brainstate.nn.AlignPostProj(
            # 连接规则：抑制性神经元向所有神经元以 2% 概率连接
            comm=brainstate.nn.EventFixedProb(
                self.n_inh, self.num, conn_num=0.02,
                conn_weight=67. * u.nS  # 抑制性突触权重更大（67 nS），平衡兴奋性
            ),
            # 突触动力学：指数衰减突触（时间常数 10 ms，典型抑制性突触特性）
            syn=brainstate.nn.Expon(self.num, tau=10. * u.ms),
            # 突触后效应：反转电位 -80 mV（抑制性，低于静息电位）
            out=brainstate.nn.COBA(E=-80. * u.mV),
            post=self.N  # 投射目标为神经元群体 N
        )

    def update(self, t):
        # 定义网络随时间的更新规则
        with brainstate.environ.context(t=t):  # 传入当前时间 t
            # 获取当前时刻的 spike 信号（布尔值数组：True 表示神经元发放）
            spk = self.N.spike.value

            # 兴奋性神经元的 spike 驱动兴奋性突触投射
            self.E(spk[:self.n_exc])  # 前 n_exc 个神经元为兴奋性

            # 抑制性神经元的 spike 驱动抑制性突触投射
            self.I(spk[self.n_exc:])  # 剩余 n_inh 个为抑制性

            # 神经元接收突触输入后更新状态，返回新的 spike 信号（输入电流为 0，仅靠突触驱动）
            spk = self.N(0. * u.nA)
            return spk
```



### 运行网络模拟
初始化网络并运行仿真，记录每个时刻的神经元 spike 活动：

```python
# 初始化 E-I 网络
net = EINet()
brainstate.nn.init_all_states(net)  # 初始化网络中所有神经元和突触的状态

# 设置仿真参数并运行
with brainstate.environ.context(dt=0.1 * u.ms):  # 时间步长 0.1 ms（高精度捕获 spike）
    # 生成仿真时间序列：从 0 ms 到 100 ms，步长为 dt
    times = u.math.arange(0. * u.ms, 100. * u.ms, brainstate.environ.get_dt())

    # 循环更新网络状态：对每个时间点调用 net.update，返回所有时刻的 spike 记录
    spikes = brainstate.compile.for_loop(
        net.update, times,
        pbar=brainstate.compile.ProgressBar(10)  # 显示进度条（总进度分 10 段）
    )
```


### 可视化网络 spike 活动
将 spike 数据绘制成“ raster plot ”，直观展示神经元在时间上的发放模式：

```python
# 提取 spike 发生的时间和神经元索引
# spikes 是二维数组（时间×神经元），True 表示该时刻该神经元发放
t_indices, n_indices = u.math.where(spikes)

# 绘制 raster plot
plt.scatter(times[t_indices], n_indices, s=1)  # s=1 控制点大小，避免重叠
plt.xlabel('Time (ms)')  # 横轴：时间
plt.ylabel('Neuron index')  # 纵轴：神经元索引（0~3999）
plt.show()
```


## 结果解读
运行代码后，你将看到一张 raster plot（ raster 图），如下图：

其中每个点代表一个神经元在特定时间的 spike 发放。典型的 E-I 网络动态具有以下特征：

**异步不规则发放**：神经元发放时间分散，无明显同步节律（符合皮层网络的静息状态）。
**稀疏活动**：大多数神经元在 100 ms 内发放次数较少（网络处于平衡状态，E-I 相互抑制）。
**无爆发式同步**：由于抑制性突触的快速反馈，避免了大规模神经元同步爆发（过度同步可能导致癫痫样活动）。

这些特征表明 E-I 网络通过兴奋性和抑制性的动态平衡，维持了稳定且符合生理特性的活动模式。


## 总结与扩展
本教程展示了如何使用 Braincell 和 brainstate 构建 E-I 神经元网络，核心要点包括：

- **层级建模**：从 HH 单神经元（`SingleCompartment`）到 E-I 网络（`DynamicsGroup`），实现从微观到宏观的整合。
- **高效仿真**：通过 `ind_exp_euler` 积分器和 `brainstate` 的并行计算支持，实现 4000 神经元网络的快速仿真。

**扩展练习**：
- 调整抑制性突触权重（如减小到 50 nS），观察网络是否出现过度同步，即爆发式活动。
- 增加兴奋性神经元比例（如 3500:500），分析 E-I 失衡对网络动态的影响。
- 延长仿真时间（如 500 ms），观察网络是否维持稳定的异步活动。

通过这些扩展，你可以深入理解 E-I 平衡在维持神经网络功能中的核心作用，以及 Braincell 在复杂网络建模中的灵活性。